In [22]:
import os

dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]
for npz_file in npz_files:
    npz_file_path = dataset_dir +"/"+ npz_file
    print(f"npz_file_path : {npz_file_path}")

npz_file_path : my_dataset/lorenz96/results_lorenz_T_1000_seed_618.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_1000_seed_169.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_1000_seed_594.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_1000_seed_584.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_1000_seed_381.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_200_seed_169.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_200_seed_584.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_1000_seed_981.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_1000_seed_671.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_200_seed_381.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_200_seed_594.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_200_seed_618.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_200_seed_671.npz
npz_file_path : my_dataset/lorenz96/results_lorenz_T_200_seed_981.npz
npz_file_path

In [21]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/"+npz_file

    # 构建命令行参数
    command = [
        "python", "main.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "250",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", os.path.join(output_dir, f"{npz_file}_output.npz"),  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = os.path.join(output_dir, f"{npz_file}_output.npz")

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: results_lorenz_T_1000_seed_618.npz
Computational Resource: cpu
node = 0
1041
Error processing results_lorenz_T_1000_seed_618.npz: Command '['python', 'main.py', '--dataset', 'lorenz', '--dsid', '1', '--model', 'sru', '--sruname', '10', '--T', '250', '--F', '10', '--nepochs', '2000', '--mu1', '0.021544', '--mu2', '0.031623', '--mu3', '0', '--lr', '0.001', '--joblog', 'output/logs/results_lorenz_T_1000_seed_618.npz_output.npz', '--inputfile', 'my_dataset/lorenz96/results_lorenz_T_1000_seed_618.npz']' returned non-zero exit status 1.
Processing file: results_lorenz_T_1000_seed_169.npz


Traceback (most recent call last):
  File "main.py", line 288, in <module>
    trainVerboseLvl)
  File "/root/autodl-tmp/SRU_for_GCI/models/sru.py", line 189, in trainSRU
    model.forward(trainingData[:,tt])
IndexError: index 54 is out of bounds for dimension 1 with size 5


Computational Resource: cpu
node = 0
1041
Error processing results_lorenz_T_1000_seed_169.npz: Command '['python', 'main.py', '--dataset', 'lorenz', '--dsid', '1', '--model', 'sru', '--sruname', '10', '--T', '250', '--F', '10', '--nepochs', '2000', '--mu1', '0.021544', '--mu2', '0.031623', '--mu3', '0', '--lr', '0.001', '--joblog', 'output/logs/results_lorenz_T_1000_seed_169.npz_output.npz', '--inputfile', 'my_dataset/lorenz96/results_lorenz_T_1000_seed_169.npz']' returned non-zero exit status 1.
Processing file: results_lorenz_T_1000_seed_594.npz


Traceback (most recent call last):
  File "main.py", line 288, in <module>
    trainVerboseLvl)
  File "/root/autodl-tmp/SRU_for_GCI/models/sru.py", line 189, in trainSRU
    model.forward(trainingData[:,tt])
IndexError: index 54 is out of bounds for dimension 1 with size 5


Computational Resource: cpu
node = 0
1041
Error processing results_lorenz_T_1000_seed_594.npz: Command '['python', 'main.py', '--dataset', 'lorenz', '--dsid', '1', '--model', 'sru', '--sruname', '10', '--T', '250', '--F', '10', '--nepochs', '2000', '--mu1', '0.021544', '--mu2', '0.031623', '--mu3', '0', '--lr', '0.001', '--joblog', 'output/logs/results_lorenz_T_1000_seed_594.npz_output.npz', '--inputfile', 'my_dataset/lorenz96/results_lorenz_T_1000_seed_594.npz']' returned non-zero exit status 1.
Processing file: results_lorenz_T_1000_seed_584.npz


...

Traceback (most recent call last):
  File "main.py", line 288, in <module>
    trainVerboseLvl)
  File "/root/autodl-tmp/SRU_for_GCI/models/sru.py", line 189, in trainSRU
    model.forward(trainingData[:,tt])
IndexError: index 54 is out of bounds for dimension 1 with size 5


Computational Resource: cpu
node = 0
1041
Error processing results_lorenz_T_500_seed_671.npz: Command '['python', 'main.py', '--dataset', 'lorenz', '--dsid', '1', '--model', 'sru', '--sruname', '10', '--T', '250', '--F', '10', '--nepochs', '2000', '--mu1', '0.021544', '--mu2', '0.031623', '--mu3', '0', '--lr', '0.001', '--joblog', 'output/logs/results_lorenz_T_500_seed_671.npz_output.npz', '--inputfile', 'my_dataset/lorenz96/results_lorenz_T_500_seed_671.npz']' returned non-zero exit status 1.
Processing file: results_lorenz_T_500_seed_981.npz


Traceback (most recent call last):
  File "main.py", line 288, in <module>
    trainVerboseLvl)
  File "/root/autodl-tmp/SRU_for_GCI/models/sru.py", line 189, in trainSRU
    model.forward(trainingData[:,tt])
IndexError: index 54 is out of bounds for dimension 1 with size 5


Computational Resource: cpu
node = 0
1041
Error processing results_lorenz_T_500_seed_981.npz: Command '['python', 'main.py', '--dataset', 'lorenz', '--dsid', '1', '--model', 'sru', '--sruname', '10', '--T', '250', '--F', '10', '--nepochs', '2000', '--mu1', '0.021544', '--mu2', '0.031623', '--mu3', '0', '--lr', '0.001', '--joblog', 'output/logs/results_lorenz_T_500_seed_981.npz_output.npz', '--inputfile', 'my_dataset/lorenz96/results_lorenz_T_500_seed_981.npz']' returned non-zero exit status 1.

All AUROC Results:


Traceback (most recent call last):
  File "main.py", line 288, in <module>
    trainVerboseLvl)
  File "/root/autodl-tmp/SRU_for_GCI/models/sru.py", line 189, in trainSRU
    model.forward(trainingData[:,tt])
IndexError: index 54 is out of bounds for dimension 1 with size 5


In [10]:
import torch

torch.cuda.empty_cache()


In [12]:
import torch

print(torch.cuda.is_available())  # 检查是否能够使用 GPU
print(torch.version.cuda)  # 查看当前 CUDA 版本
print(torch.cuda.current_device())  # 查看当前设备的 ID


True
10.0.130
0
